## Data Exploration and Analysis

1.1 Install necessary libraries

In [1]:
!pip install matplotlib seaborn opencv-python scikit-learn tensorflow

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/a4/7d/f1c30a92854540bf789e9cd5dde7ef49bbe63f855b85a2e6b3db8135c591/opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/3c/e3/e868f1d5951047f950d2ba1e04a765a3328a51f06996b67976d6102f8227/tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/b8/25/155f9f080d5e4bc0082edfda032ea2bc2b8fab3f4d25d46c1e9dd22a1a89/flatbuffers-25.2.10-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/a3/61

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\binwi\\anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [14]:
conda install numpy=1.21

^C

Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda update matplotlib

1.2 Load and Explore the Dataset

In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import pandas as pd

# Define paths to the folders containing the dataset
first_print_path = "assignment_data/first_print"
second_print_path = "assignment_data/second_print"

# Load images and labels
def load_images(path, label):
    images = []
    labels = []
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        img = np.array(img)
        images.append(img)
        labels.append(label)
    return images, labels

first_print_images, first_print_labels = load_images(first_print_path, 0)  # 0 for first print (original)
second_print_images, second_print_labels = load_images(second_print_path, 1)  # 1 for second print (counterfeit)

# Combine all images and labels
images = first_print_images + second_print_images
labels = first_print_labels + second_print_labels

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Show some example images from both classes
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Display first print (original)
ax[0].imshow(images[0], cmap='gray')
ax[0].set_title('First Print (Original)')
ax[0].axis('off')

# Display second print (counterfeit)
ax[1].imshow(images[len(first_print_images)], cmap='gray')
ax[1].set_title('Second Print (Counterfeit)')
ax[1].axis('off')

plt.show()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\binwi\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\binwi\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\binwi\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "C:\Users\binwi\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

1.2 Dataset statistics

In [ ]:
# Dataset statistics
print(f"Total number of images: {len(images)}")
print(f"Number of first prints (original): {len(first_print_images)}")
print(f"Number of second prints (counterfeit): {len(second_print_images)}")
print(f"Image shape: {images[0].shape}")


## Step 2: Feature Engineering

### 2.1 Basic image pre-processing and feature extraction
You can consider features like histogram of oriented gradients (HOG) or other simple texture features.

In [ ]:
from skimage.feature import hog
from skimage import exposure

def extract_hog_features(images):
    hog_features = []
    for img in images:
        fd, _ = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
        hog_features.append(fd)
    return np.array(hog_features)

# Extract HOG features
hog_features = extract_hog_features(images)

# Show one of the extracted HOG features
fd, hog_image = hog(images[0], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
plt.imshow(hog_image, cmap=plt.cm.gray)
plt.title("HOG Feature Visualization")
plt.show()


## Step 3: Model Development

### 3.1 Train-Test Split

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.2, random_state=42)


### 3.2 Traditional Machine Learning Model (SVM)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Train SVM model
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Predict on test set
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM model
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm)}")
print(f"SVM Classification Report:\n{classification_report(y_test, y_pred_svm)}")
print(f"SVM Confusion Matrix:\n{confusion_matrix(y_test, y_pred_svm)}")


### 3.3 Deep Learning Model (CNN)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array

# Preprocess images for CNN
def preprocess_images_for_cnn(images):
    return np.array([img_to_array(img) for img in images])

X_cnn = preprocess_images_for_cnn(images)
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, labels, test_size=0.2, random_state=42)

# Build CNN model
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(X_cnn[0].shape)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))  # Binary classification

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train CNN model
cnn_model.fit(X_train_cnn, np.array(y_train_cnn), epochs=10, batch_size=32, validation_data=(X_test_cnn, np.array(y_test_cnn)))

# Evaluate CNN model
cnn_loss, cnn_acc = cnn_model.evaluate(X_test_cnn, np.array(y_test_cnn))
print(f"CNN Accuracy: {cnn_acc}")


## Step 4: Evaluation and Results

### 4.1 Confusion Matrix and Classification Report for Both Models

In [ ]:
# SVM model evaluation
y_pred_svm = svm_model.predict(X_test)
print("SVM Model Evaluation")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_svm)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred_svm)}")

# CNN model evaluation
y_pred_cnn = (cnn_model.predict(X_test_cnn) > 0.5).astype("int32")
print("\nCNN Model Evaluation")
print(f"Accuracy: {accuracy_score(y_test_cnn, y_pred_cnn)}")
print(f"Classification Report:\n{classification_report(y_test_cnn, y_pred_cnn)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test_cnn, y_pred_cnn)}")
